# Elastic Search
# 1. 검색 시스템 이해하기
- 자세한 내용은 "엘라스틱서치 실무가이드" 교재 참고

## 모든 프로그램은 C드라이브에 설치하여서 진행 ###

## elasticsearch 실행
- elasticsearch.yml 파일 수정
### elasticsearch.yml ## (p.17)
cluster.name: javacafe-cluster
node.name: javacafe-node1
network.host: 0.0.0.0
http.port: 9200
transport.tcp.port: 9300
node.master: true
node.data: true
path.repo: ["C:/book_backup/search_example","C:/book_backup/agg_example"]
(window용)

- bin> 폴더에 elasticsearch.bat cmd로 실행

## Data Loading하기
### 1. book_backup/search_example (data 읽기~로딩)
--------------------윈도우-----------------
<br>
curl -H "Content-Type: application/json" -XPUT "http://localhost:9200/_snapshot/javacafe" -d "{\"type\": \"fs\", \"settings\": {\"location\":\"C:/book_backup/search_example\", \"compress\": true}}"
<br>
curl -XGET "http://localhost:9200/_snapshot/javacafe/_all"?pretty -u elastic:bigdata
<br>
curl -XPOST "http://localhost:9200/_snapshot/javacafe/movie-search/_restore" -u elastic:bigdata
<br>
--------------------리눅스-----------------
<br>
curl -H "Content-Type: application/json" -XPUT "http://localhost:9200/_snapshot/javacafe" -d "{\"type\": \"fs\", \"settings\": {\"location\":\"/home/elastic-2/book_backup/search_example\", \"compress\": true}}" -u elastic:bigdata
<br>
curl -XGET "http://localhost:9200/_snapshot/javacafe/_all"?pretty -u elastic:bigdata
<br>
curl -XPOST "http://localhost:9200/_snapshot/javacafe/movie-search/_restore" -u elastic:bigdata
<br>
<br>

### 2. book_backup/agg_example (data 읽기~로딩)

--------------------윈도우-----------------
<br>
curl -H "Content-Type: application/json" -XPUT "http://localhost:9200/_snapshot/apache-web-log" -d "{\"type\": \"fs\", \"settings\": {\"location\":\"C:/book_backup/agg_example\", \"compress\": true}}"
<br>
curl -XGET "http://localhost:9200/_snapshot/apache-web-log/_all" -u elastic:bigdata
<br>
curl -XPOST "http://localhost:9200/_snapshot/apache-web-log/default/_restore" -u elastic:bigdata
<br>
--------------------리눅스-----------------
<br>
curl -H "Content-Type: application/json" -XPUT "http://localhost:9200/_snapshot/apache-web-log" -d "{\"type\": \"fs\", \"settings\": {\"location\":\"/home/elastic-2/book_backup/agg_example\", \"compress\": true}}" -u elastic:bigdata
<br>
curl -XGET "http://localhost:9200/_snapshot/apache-web-log/_all" -u elastic:bigdata
<br>
curl -XPOST "http://localhost:9200/_snapshot/apache-web-log/default/_restore" -u elastic:bigdata
<br>
<br>
<br>

## kibana 실행
- kibana.yml 파일에서 아래 내용 주석 제거
#elasticsearch.url: "http://localhost:9200"
- bin 폴더에 kibana.bat cmd로 실행
- http://localhost:5601에서 실행 되었는지 확인
- Dev Tools > query 작성

## Data 입력
PUT movie_kibana_execute/_doc/1
{
  "message":"helloworld"
}

## Data 입력 확인
GET movie_kibana_execute/_search
{
  "query": {
    "match_all": {}
  }
}

## 인덱스 API 관리 (p.42)
### 인덱스 생성
PUT /movie
{
  "settings": {
    "number_of_shards": 3,
    "number_of_replicas": 2
  },
  "mappings": {
    "_doc": {
      "properties":{
        "movieCd" : { "type" : "integer"},
        "movieNm" : { "type" : "text"},
        "movieNmEn" : { "type" : "text"},
        "prdtYear" : { "type" : "integer"},
        "openDt" : { "type" : "date"},
        "typeNm" : { "type" : "keyword"},
        "prdtStatNm" : { "type" : "keyword"},
        "nationAlt" : { "type" : "keyword"},
        "genreAlt" : { "type" : "keyword"},
        "repNationNm" : { "type" : "keyword"},
        "repGenreNm" : { "type" : "keyword"}
      }
    }
  }
}
- 실행 결과
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "movie"
}

### 인덱스 삭제
DELETE /movie

- 실행 결과
{
  "acknowledged": true
}

## 문서 관리 API
### 문서 생성
POST /movie/_doc/1
{
  "movieCd" : "1",
  "movieNm" : "살아남은 아이",
  "movieNmEn" : "Last Child",
  "prdtYear" : "2017",
  "openDt" : "2017-10-20",
  "typeNm" : "장편",
  "prdtStatNm" : "기타",
  "nationAlt" : "한국",
  "genreAlt" : "드라마, 가족",
  "repNationNm" : "한국",
  "repGenreNm" : "드라마"
}
- 실행 결과
{
  "_index": "movie",
  "_type": "_doc",
  "_id": "1",
  "_version": 1,
  "result": "created",
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 0,
  "_primary_term": 1
}
- ID ("1") 지정하지 않으면 무작위 이름 생성

### 문서 조회
GET /movie/_doc/1
- 실행결과
{
  "_index": "movie",
  "_type": "_doc",
  "_id": "1",
  "_version": 1,
  "found": true,
  "_source": {
    "movieCd": "1",
    "movieNm": "살아남은 아이",
    "movieNmEn": "Last Child",
    "prdtYear": "2017",
    "openDt": "2017-10-20",
    "typeNm": "장편",
    "prdtStatNm": "기타",
    "nationAlt": "한국",
    "genreAlt": "드라마, 가족",
    "repNationNm": "한국",
    "repGenreNm": "드라마"
  }
}

### 문서 삭제
DELETE /movie/_doc/1
- 실행 결과
{
  "_index": "movie",
  "_type": "_doc",
  "_id": "1",
  "_version": 2,
  "result": "deleted",
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 1,
  "_primary_term": 1
}

## 검색 API
### URL 방식
GET /movie_search/_doc/_search?q=prdtYear:2017&pretty=true
{
  "sort" : {
    "movieCd" : {
      "order" : "asc"
    }
  }
}

### Request body 방식 : 조건문 표현이 더 가능하기에 많이 사용하는 방식
POST /movie_search/_search
{
  "query": {
    "term": {
      "typeNm": "장편"
    }
  }
}